In [1]:
# load the vector files
import sys
import io
setting = "p2v"

vectorFilesDir = "../Data/vectors/"+setting+"/"+setting+".txt"
allPaperVectors = []

with open(vectorFilesDir, 'r', encoding = 'utf8') as f:
    for line in f:
        read_data = line.split(" ")
        paper_Vectors = read_data
        allPaperVectors.append(paper_Vectors)
f.close()
        
print("Total vector records:",len(allPaperVectors))
print(allPaperVectors[0])

Total vector records: 3149758
['22516865', '0.0422827', '-0.0443959', '-0.00631522', '-0.0297457', '-0.203585', '0.0791923', '-0.156993', '0.102914', '-0.112826', '0.0301516', '-0.118142', '-0.0447324', '0.0131143', '0.127706', '0.107417', '-0.0843709', '-0.0808756', '-0.0504176', '0.132817', '0.0157529', '-0.0383218', '0.165365', '-0.0619297', '-0.0345336', '0.110633', '0.0860297', '-0.0835764', '0.193302', '0.015641', '0.0323645', '0.00893256', '0.151895', '0.0534986', '-0.18639', '-0.0558457', '-0.0603591', '-0.00530632', '-0.000749852', '0.0772172', '0.0646557', '-0.106445', '-0.028214', '0.0400324', '0.11686', '-0.153841', '-0.064435', '-0.0270715', '-0.0234374', '0.0588498', '-0.0750828', '-0.107259', '-0.0351021', '-0.0166648', '-0.135551', '0.1277', '-0.118618', '-0.0417162', '0.0543777', '0.129501', '-0.152003', '0.0624739', '0.175164', '0.0680417', '-0.0896099', '0.171857', '-0.0123408', '-0.0424403', '-0.017759', '0.0631562', '-0.0947245', '-0.0297129', '-0.0952609', '0.0229

In [12]:
import re
import numpy as np
import os

# collect data
fileDir = "../Data/filteredSameNameAuthor/filter=30/"
fileList = os.listdir(fileDir)
fileList.sort()
print(fileList)


['chung-may yang.txt', 'chung-may yang0.txt', 'chung-may yang1.txt', 'david g lloyd.txt', 'david g lloyd0.txt', 'david g lloyd1.txt', 'jeong hwan kim.txt', 'jeong hwan kim0.txt', 'jeong hwan kim1.txt', 'kevin m. ryan.txt', 'kevin m. ryan0.txt', 'kevin m. ryan1.txt', 'lei wang.txt', 'lei wang0.txt', 'lei wang1.txt', 'michael wagner.txt', 'michael wagner0.txt', 'michael wagner1.txt']


In [13]:
# remove author(positive sample) from other(negative sample)
import random
def extractNegativeSample(positiveSample, allSample):
    negativeSample = [x for x in allSample if x not in positiveSample]
    print("Total negative sample size:", len(negativeSample))
    return negativeSample

In [14]:
# collect class vectors
import pandas as pd
import numpy as np

def extractVectors(author_pids, NegativeSample_pid, allPaperVectors):
    # extract class one vectors
    author_features = []
    for pid in author_pids:
         for paper_Vectors in allPaperVectors:
            if(paper_Vectors[0] == pid):
                author_features.append(paper_Vectors)
    print("Positive sample size: ", len(author_features))
    classOne = pd.DataFrame(author_features)
    classOne["label"] = 0
    # extract class two vectors
    other_features = []
    for pid in NegativeSample_pid:
        for paper_Vectors in allPaperVectors:
            if(paper_Vectors[0] == pid):
                other_features.append(paper_Vectors)
    print("Negative sample size: ", len(other_features))
    classTwo = pd.DataFrame(other_features)
    classTwo["label"] = 1
    return classOne, classTwo


In [15]:
# combine data from different class get all data
def combineClassesData(classOne,classTwo):
    combinedData = pd.concat([classOne, classTwo])
    combinedData = combinedData.sample(frac=1).reset_index(drop=True)
    # take the paper id out
    paperID = combinedData[0]
    # split data and label
    data = combinedData.drop([0,'label'], axis=1)
    label = combinedData['label']
    print("Total sample size and shape: ",data.shape)
    return data, label, paperID

In [16]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import (precision_score, recall_score,f1_score,accuracy_score)
# cross validation
def k_fold_cv(author, data, label, classifier, k=10):
    kf = KFold(n_splits=k, shuffle=False)
    allTrueLabel = []
    allPredLabel = []
    for counter,(train_index, test_index) in enumerate(kf.split(data)):
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # split train and test
        data_train, data_test = data.iloc[train_index], data.iloc[test_index]
        label_train, test_true_label = label.iloc[train_index], label.iloc[test_index]
        # fit data to svm
        classifier.fit(data_train, label_train)
        # get predicted label
        label_pred = classifier.predict(data_test)
        allTrueLabel.extend(test_true_label)
        allPredLabel.extend(label_pred)
#         # get predict proba
#         proba = classifier.predict_proba(data_test)
        # find out which sample cause the issue
        print("Pred: ",label_pred)
        print("True: ", test_true_label.values.tolist())
        print("Mislabeled sample: ",end='')
        for i in range(len(test_true_label)):
            if(label_pred[i]!=test_true_label[test_index[i]]):
                print(paperID[test_index[i]]+",",end='')
        print()
        # print("True positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
        # .format(tp=round_tp, fp=round_fp, fn=round_fn, tn=round_tn))

    accuracy = accuracy_score(allTrueLabel, allPredLabel)
    f1 = f1_score(allTrueLabel, allPredLabel,average='binary')
    precision = precision_score(allTrueLabel, allPredLabel)
    recall = recall_score(allTrueLabel, allPredLabel)
    tn,fp,fn,tp = metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel()
    
    print("Author: ", author)
    print("Classifier: ",classifier)
    print(metrics.classification_report(allTrueLabel, allPredLabel))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel())
    print("Accuracy: ",accuracy)
    print("F1: ", f1)
    print("Precision: ", precision)
    print("Recall: ", recall)
    
    return accuracy, f1, precision, recall, tn, fp, fn, tp
    # return ppv, npv, specificity, sensitivity, accuracy, f1proba = linear_svc.predict_proba(allDatas)

In [17]:
from sklearn import svm
from sklearn import linear_model
from sklearn.decomposition import PCA

# loop through files in directory |
# add name to name list
# author as positive sample, other as all samples
name_list = []
# create name list for all authors have same name
for file in fileList:
    if not file.startswith('.'):
        if not re.match(r'\D*\d+.txt$', file):
            # fix the coding issue
            name_list.append(file.encode("utf-8", "surrogateescape").decode('utf8','surrogateescape')[:-4])
# print(name_list)

# loop through all the author and gather result
allauthor = []
authorSampleSize = []
allSampleSize = []
allaccuracy = []
allf1 = []
allprecision = []
allrecall = []
alltn = []
allfp = []
allfn = []
alltp = []

for name in name_list:
    other_pids = []
    # read other sample
    with open((fileDir+name+".txt").encode('utf-8'), 'r', encoding = 'utf8') as f:
        for line in f:
            other_pids.extend(line.strip().split(" "))
#     print(name)
    for file in fileList:
        file=file.encode("utf-8", "surrogateescape").decode('utf8','surrogateescape')
        if not file.startswith('.'):
            if re.match(r'\D*\d+.txt$', file):
                if name in file:
                    print(os.path.splitext(file)[0])
                    # add author to list for final output
                    allauthor.append(os.path.splitext(file)[0])
                    author_pids = []
                    # read author sample
                    with open((fileDir+os.path.splitext(file)[0]+".txt").encode('utf-8'), 'r', encoding = 'utf8') as f:
                        for line in f:
                            author_pids.extend(line.strip().split(" "))
                    # print properties
                    authorSampleSize.append(len(author_pids))
                    allSampleSize.append(len(other_pids))
                    print(len(author_pids))
                    print(len(other_pids))
                    # remove author(positive sample) from other(all sample) to create negative sample
                    NegativeSample_pid = extractNegativeSample(author_pids, other_pids)
                    print(len(NegativeSample_pid))
                    # collect all vector
                    classOne, classTwo = extractVectors(author_pids,NegativeSample_pid,allPaperVectors)
                    print(classOne.shape)
                    print(classTwo.shape)
                    # combine data from different class get all data
                    data, label, paperID = combineClassesData(classOne, classTwo)
#                     # PCA transform
#                     pca = PCA(n_components=2)
#                     pca_transformed = pd.DataFrame(pca.fit_transform(X=data, y=label))
#                     pca_transformed["label"] = label
#                     # 10 fold cv SVM linear kernel
#                     linear_svc = svm.SVC(kernel='linear', class_weight='balanced', probability=True,cache_size=4000)
#                     accuracy, f1, precision, recall, tn, fp, fn, tp= k_fold_cv(os.path.splitext(file)[0],data, label, linear_svc,10)
#                     # 10 fold cv SVM rbf kernel
#                     rbf_svc = svm.SVC(kernel='rbf')
#                     accuracy, f1, precision, recall, tn, fp, fn, tp = k_fold_cv(os.path.splitext(file)[0],data, label, rbf_svc,10)
                    # 10 fold cv logistic regression
                    logistic = linear_model.LogisticRegression()
                    accuracy, f1, precision, recall, tn, fp, fn, tp = k_fold_cv(os.path.splitext(file)[0],data, label, logistic,10)
                    allaccuracy.append(accuracy)
                    allf1.append(f1)
                    allprecision.append(precision)
                    allrecall.append(recall)
                    alltn.append(tn)
                    alltp.append(tp)
                    allfn.append(fn)
                    allfp.append(fp)
# write evaluation result to excel
output = pd.DataFrame({'author':allauthor,"AuthorSampleSize":authorSampleSize,
                       "accuracy":allaccuracy,"f1":allf1, "precision":allprecision,
                      "recall":allrecall, "AllSameNameSampleCount":allSampleSize,
                      "True positive": alltp, "True negative":alltn,
                      "False positive": allfp, "False negative": allfn})
filename = "logistic_regression_"+setting+"_filter=10.csv"
output.to_csv("../result/"+filename, encoding='utf-8',index=False)
print("Done")

chung-may yang0
42
113
Total negative sample size: 71
71
Positive sample size:  42
Negative sample size:  71
(42, 102)
(71, 102)
Total sample size and shape:  (113, 100)
Pred:  [1 1 1 1 1 1 1 1 1 1 1 1]
True:  [1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0]
Mislabeled sample: 8282044,26732884,15389271,16858434,20039856,24296307,
Pred:  [1 0 1 1 1 1 1 1 1 1 1 0]
True:  [1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Mislabeled sample: 27084002,26582311,15621160,1547971,
Pred:  [1 1 1 0 1 1 1 0 1 1 1 1]
True:  [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
Mislabeled sample: 20688740,26311257,24107645,
Pred:  [1 1 1 1 1 1 1 1 1 1 1]
True:  [1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1]
Mislabeled sample: 16139013,26047532,16159720,
Pred:  [1 1 1 1 1 1 1 1 1 1 1]
True:  [0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1]
Mislabeled sample: 14767657,19369181,17721499,26891760,
Pred:  [1 1 1 1 1 1 1 1 1 1 1]
True:  [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1]
Mislabeled sample: 24743637,18347622,19394702,24669373,25572579,
Pred:  [1 1 0 1 1 1 1 1 1 1 1]
True:  [1

KeyboardInterrupt: 

In [18]:
# hard code to read the file one by one
# store the features for classification
author_pids = []
other_pids = []
name = "lei wang0"
# author as positive sample, other as all samples
with open(fileDir+name+".txt", 'r', encoding = 'utf8') as f:
    for line in f:
        author_pids.extend(line.strip().split(" "))

with open(fileDir+"lei wang.txt", 'r', encoding = 'utf8') as f:
    for line in f:
        other_pids.extend(line.strip().split(" "))
        
# size of each class
print(len(author_pids))
print(len(other_pids))

53
117


In [19]:
# extract negative Sample
NegativeSample_pid = extractNegativeSample(author_pids, other_pids)
print("Choicen negative sample ", len(NegativeSample_pid))

Total negative sample size: 64
Choicen negative sample  64


In [20]:
classOne, classTwo = extractVectors(author_pids,NegativeSample_pid,allPaperVectors)
print(classOne.shape)
print(classTwo.shape)

Positive sample size:  53
Negative sample size:  64
(53, 102)
(64, 102)


In [21]:
data, label, paperID = combineClassesData(classOne, classTwo)

Total sample size and shape:  (117, 100)


In [22]:
from sklearn import svm
# create linear SVM model
linear_svc = svm.SVC(kernel='linear', class_weight='balanced', probability=True,cache_size=4000)

accuracy, f1, precision, recall, tn, fp, fn, tp= k_fold_cv(os.path.splitext(file)[0],data, label, linear_svc,10)
#print(linear_svc)
# # fit model and do 10-fold cv
# k_fold_cv(name,data, label, linear_svc,10)

# # get number of support vectors for each class
# print(linear_svc.n_support_)

# '''
# # compute the distance to decision boundry (Not same as confidence measure)
# Distance = linear_svc.decision_function(allDatas)

# # computer the confidence measure (Platt scaling: transforming the outputs of a 
# # classification model into a probability distribution over classes)
# # P(class/input) = 1 / (1 + exp(A * f(input) + B))
# # P(class/input) is the probability that “input” belongs to “class” 
# # and f(input) is the signed distance of the input datapoint from the boundary,
# # which is basically the output of “decision_function”. 

# proba = linear_svc.predict_proba(allDatas)

# '''

# # create rbf SVM model with C=10 where (C*Error) is added into minimize function
# # C big means error matter more
# rbf_svc = svm.SVC(kernel='rbf',probability=True)

# # fit model and do 10-fold cv
# k_fold_cv(name,data, label, rbf_svc)

# # get number of support vectors for each class
# print(rbf_svc.n_support_)


Pred:  [1 1 1 1 0 0 1 1 1 0 1 1]
True:  [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1]
Mislabeled sample: 
Pred:  [1 0 0 0 0 1 1 1 0 1 0 0]
True:  [1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0]
Mislabeled sample: 
Pred:  [1 0 1 1 1 1 0 0 1 1 1 0]
True:  [1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0]
Mislabeled sample: 
Pred:  [1 1 1 1 0 1 0 1 1 0 1 1]
True:  [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1]
Mislabeled sample: 
Pred:  [1 0 1 1 0 0 0 0 1 0 0 1]
True:  [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1]
Mislabeled sample: 
Pred:  [0 1 1 0 1 0 0 1 0 1 0 1]
True:  [0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
Mislabeled sample: 22751827,
Pred:  [1 1 1 0 1 0 0 0 0 0 1 0]
True:  [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
Mislabeled sample: 
Pred:  [1 0 1 0 1 0 1 0 0 1 1]
True:  [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]
Mislabeled sample: 
Pred:  [1 1 0 1 1 1 0 1 0 0 1]
True:  [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1]
Mislabeled sample: 
Pred:  [1 0 0 1 0 0 0 0 0 0 1]
True:  [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
Mislabeled sample: 
Author:  david g lloyd1
Classifier: 